In [1]:
from pyomo.environ import TerminationCondition, value
from hda_flowsheet import FS

In [3]:
FS.run_steps(before="solve_optimization")

TypeError: BaseFlowsheetRunner.run_steps() got an unexpected keyword argument 'before'

In [3]:
m = FS.model

# What is the total operating cost?
print("operating cost = $", value(m.fs.operating_cost))

# For this operating cost, what is the amount of benzene we are able to produce and what purity we are able to achieve?
m.fs.F102.report()
print()
print("benzene purity = ", value(m.fs.purity))


# How much benzene are we losing in the F101 vapor outlet stream?
from idaes.core.util.tables import (
    create_stream_table_dataframe,
    stream_table_dataframe_to_string,
)
st = create_stream_table_dataframe({"Reactor": m.fs.s05, "Light Gases": m.fs.s06})
print(stream_table_dataframe_to_string(st))

operating cost = $ 419122.3387677973

Unit : fs.F102                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key             : Value       : Units  : Fixed : Bounds
          Heat Duty :      7352.5 :   watt : False : (None, None)
    Pressure Change : -2.0000e+05 : pascal :  True : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                                Units         Inlet    Vapor Outlet  Liquid Outlet
    flow_mol_phase_comp ('Liq', 'benzene')   mole / second    0.20460   1.0000e-08      0.062620  
    flow_mol_phase_comp ('Liq', 'toluene')   mole / second   0.062520   1.0000e-08      0.032257  
    flow_mol_phase_comp ('Liq', 'methane')   mole / second 2.6712e-07   1.0000e-08    9.4877e-08  
    flow_mol_phase_comp ('Liq', 'hydrogen')  mole / second 2.6

In [4]:
FS.run_steps(last="solve_optimization")

2025-11-23 11:56:06 [INFO] idaes.init.fs.H101.control_volume.properties_in: Initialization Complete
2025-11-23 11:56:06 [INFO] idaes.init.fs.H101.control_volume.properties_out: Initialization Complete
2025-11-23 11:56:06 [INFO] idaes.init.fs.R101.control_volume.properties_in: Initialization Complete
2025-11-23 11:56:06 [INFO] idaes.init.fs.R101.control_volume.properties_out: Initialization Complete
2025-11-23 11:56:06 [INFO] idaes.init.fs.F101.control_volume.properties_in: Initialization Complete
2025-11-23 11:56:06 [INFO] idaes.init.fs.F101.control_volume.properties_out: Initialization Complete
2025-11-23 11:56:06 [INFO] idaes.init.fs.S101.mixed_state: Initialization Complete
2025-11-23 11:56:06 [INFO] idaes.init.fs.S101.purge_state: Initialization Complete
2025-11-23 11:56:06 [INFO] idaes.init.fs.S101.recycle_state: Initialization Complete
2025-11-23 11:56:06 [INFO] idaes.init.fs.S101: Initialization Step 2 Complete: optimal - <undefined>
2025-11-23 11:56:06 [INFO] idaes.init.fs.F102

In [6]:
m = FS.model
results = FS.results

assert results.solver.termination_condition == TerminationCondition.optimal

print("operating cost = $", value(m.fs.operating_cost))

print()
print("Product flow rate and purity in F102")

m.fs.F102.report()

print()
print("benzene purity = ", value(m.fs.purity))

print()
print("Overhead loss in F101")
m.fs.F101.report()


# assert value(m.fs.operating_cost) == pytest.approx(312786.338, abs=1e-3)
# assert value(m.fs.purity) == pytest.approx(0.818827, abs=1e-3)

print("Optimal Values")
print()

print("H101 outlet temperature = ", value(m.fs.H101.outlet.temperature[0]), "K")

print()
print("R101 outlet temperature = ", value(m.fs.R101.outlet.temperature[0]), "K")

print()
print("F101 outlet temperature = ", value(m.fs.F101.vap_outlet.temperature[0]), "K")

print()
print("F102 outlet temperature = ", value(m.fs.F102.vap_outlet.temperature[0]), "K")
print("F102 outlet pressure = ", value(m.fs.F102.vap_outlet.pressure[0]), "Pa")

# assert value(m.fs.H101.outlet.temperature[0]) == pytest.approx(500, abs=1e-3)
# assert value(m.fs.R101.outlet.temperature[0]) == pytest.approx(696.112, abs=1e-3)
# assert value(m.fs.F101.vap_outlet.temperature[0]) == pytest.approx(301.878, abs=1e-3)
# assert value(m.fs.F102.vap_outlet.temperature[0]) == pytest.approx(362.935, abs=1e-3)
# assert value(m.fs.F102.vap_outlet.pressure[0]) == pytest.approx(105000, abs=1e-2)

operating cost = $ 312786.3383406732

Product flow rate and purity in F102

Unit : fs.F102                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key             : Value       : Units  : Fixed : Bounds
          Heat Duty :      8377.0 :   watt : False : (None, None)
    Pressure Change : -2.4500e+05 : pascal : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                                Units         Inlet    Vapor Outlet  Liquid Outlet
    flow_mol_phase_comp ('Liq', 'benzene')   mole / second    0.21743   1.0000e-08      0.067425  
    flow_mol_phase_comp ('Liq', 'toluene')   mole / second   0.070695   1.0000e-08      0.037507  
    flow_mol_phase_comp ('Liq', 'methane')   mole / second 2.8812e-07   1.0000e-08    1.0493e-07  
    flow_mol_phase_comp 

In [7]:
FS.show_diagram()

```mermaid
flowchart LR
    Unit_B["M101"]
    Unit_C["H101"]
    Unit_D["R101"]
    Unit_E["F101"]
    Unit_F["S101"]
    Unit_G["C101"]
    Unit_H["F102"]
    Unit_B --> Unit_C
    Unit_C --> Unit_D
    Unit_D --> Unit_E
    Unit_E --> Unit_F
    Unit_F --> Unit_G
    Unit_G --> Unit_B
    Unit_E --> Unit_H

```

Degrees of freedom: 5

fs                        : 5
fs.C101                   : 0
fs.C101.control_volume    : 11
fs.F101                   : 1
fs.F101.control_volume    : 11
fs.F101.split             : 0
fs.F102                   : 2
fs.F102.control_volume    : 12
fs.F102.split             : 0
fs.H101                   : 1
fs.H101.control_volume    : 11
fs.M101                   : 0
fs.R101                   : 1
fs.R101.control_volume    : 12
fs.S101                   : -10
fs.reaction_params        : 0
fs.thermo_params          : 0
fs.thermo_params.Liq      : 0
fs.thermo_params.Vap      : 0
fs.thermo_params.benzene  : 0
fs.thermo_params.hydrogen : 0
fs.thermo_params.methane  : 0
fs.thermo_params.toluene  : 0
